Gauss elimination without pivoting

In [14]:
import time
import random
import pandas as pd
import math
from tqdm import tqdm
from copy import copy, deepcopy
from pprint import pprint
from functools import partial
from matplotlib import pyplot as plt

<h> Q1 </h>

(i) Ans:

(ii) Ans:

Gaussian elimination code:

In [21]:
'''Note: all the matrices and vectors are implemented in python lists'''

# function to round off a value to d significant digits (dS arithmetic)
def roundto_dS(val, d: int = 5):
    if val == 0:
        rounded_val = 0
    else:
        rounded_val = round(val, d - int(math.floor(math.log10(abs(val)))) - 1)
    return rounded_val

# function to check if the given matrix is a square matrix or not
def is_square_matrix(matrix: list):
    is_square = 1
    for row in matrix:
        if len(row) != len(matrix):
            is_square = 0
    return is_square

# function to return the rank of a matrix
def rank(matrix: list):
    rank = 0
    for row in matrix:
        if sum(row) != 0:
            rank += 1
    return rank


### function for gauss elimination ###
'''Note: Consider a linear system with n equations and n unknowns.'''
# A: n x n square matrix (coefficient matrix)
# b: vector of size n
# d: number of significant digits used for each operation (dS arithmetic)
# pivoting: perform partial pivoting if True

def gauss_elimination(A: list, b: list, d: int = 5, pivoting: bool = True):
    if len(A) != len(b):
        print('Number of rows in A is not equal to the length of b. Not a linear system.')
        return
    elif is_square_matrix(A) == 0:
        print('A is not a square matrix.')
        return
    else:
        ### implement the gauss elimination here. ###
        
        n = len(A)  # number of rows (n) in matrix A
        
        # create a dictionary to output the values of unknown parameters x0, x1, ...
        # and initialize None for all the varirables.
        # output format is like {'x0': val1, 'x1': val2, ...}
        output_vals = {}
        for xi in range(n):
            output_vals['x'+str(xi)] = None
        
        # create augmented matrix
        Aug = A
        for row_index in range(n):
            Aug[row_index].append(b[row_index])
        
        
        # 1. forward elimination (transform the augmented matrix into REF)
        
        # iterate through the rows.
        for row_index in range(n):
            
            # do partial pivoting if pivoting is True.
            if pivoting:
                # initialize max value in column.
                max_in_col = abs(Aug[row_index][row_index])
                max_row_index = row_index
                # search for the max value in column. here ri2 is short for row_index2.
                for ri2 in range(row_index + 1, n):
                    if abs(Aug[ri2][row_index]) > max_in_col:
                        max_in_col = abs(Aug[ri2][row_index])
                        max_row_index = ri2
                # perform partial pivoting if max value is not in the current row.
                if max_row_index != row_index:
                    temp = Aug[row_index]
                    Aug[row_index] = Aug[max_row_index]
                    Aug[max_row_index] = temp
            
            # row transform. here ri2 is short for row_index2.
            for ri2 in range(row_index + 1, n):
                # check if the value is already zero and skip for zero.
                if Aug[ri2][row_index] == 0:
                    continue
                
                # calculate the multiplier with the index row
                multiplier = roundto_dS(-Aug[ri2][row_index]/Aug[row_index][row_index], d)
                
                # assign Aug[ri2][row_index] to zero. Prior values are already assigned to zero
                # in the previous cycles of the loop.
                Aug[ri2][row_index] = 0
                
                # compute the values in row. Iterate through the columns from row_index+1 to n+1
                # since this is augmented matrix, there are n+1 columns.
                for col_index in range(row_index + 1, n + 1):
                    Aug[ri2][col_index] = roundto_dS(Aug[ri2][col_index] + multiplier * Aug[row_index][col_index], d)

        
        # 2. back substitution
        
        # get the REF(A) from the transformed augmented matrix.
        REF_A = A
        for row_index in range(n):
            for col_index in range(n):
                REF_A[row_index][col_index] = Aug[row_index][col_index]
        
        # determine if the given linear system has solutions.
        
        # a. No solution.
        if rank(REF_A) != rank(Aug):
            print('The system is inconsistent. No solutions.')
            return
        
        # b. Infinitely many solutions.
        elif rank(REF_A) < n:
            print('The system has infinitely many solutions.')
            return
        
        # c. Unique solution. (when rank(REF_A)==n)
        else:
            # iterate through the rows backward
            for row_index in range(n-1,-1,-1):
                sum = 0  # variable to keep the sum of values with solved parameters.
                
                # iterate through the columns to get sum of values.
                for col_index in range(n-1,row_index,-1):
                    sum = roundto_dS(sum + Aug[row_index][col_index]*output_vals['x'+str(col_index)], d)
                
                output_vals['x'+str(row_index)] = roundto_dS((Aug[row_index][n] - sum)/Aug[row_index][row_index], d)
            
            return output_vals

In [22]:
gauss_elimination(A=[[6, 1, -6, -5], [2, 2, 3, 2], [4, -3, 0, 1], [0, 2, 0, 1]], b=[6, -2, -7, 0], pivoting=True, d=19)

{'x0': -0.5,
 'x1': 1.0000000000000002,
 'x2': 0.3333333333333337,
 'x3': -2.0000000000000004}

In [7]:
for i in range(2,2,-1):
    print(i)